## "서울시 지하철호선별 역별 승하차 인원 정보 데이터" 전처리 과정
- 해당 데이터를 가공하여 연습문제를 푸는 것이 목표 입니다.
- 다양한 데이터를 통한 반복 연습으로 데이터 전처리에 더욱 능숙해질 수 있도록 하겠습니다.
- EDA를 통해 데이터를 이해하기 위한 전 단계입니다.

### 실습 데이터 안내
- 지하철 승하차 이용객 데이터 : 2022.01.01 ~ 2022.12.31 
- 서울 열린데이터 광장 : http://data.seoul.go.kr/dataList/OA-12914/A/1/datasetView.do
- "CARD_SUBWAY_MONTH_2022.csv" 파일을 다운로드 받아 진행하면 됩니다.

## 데이터 전처리 실습을 통해 해결해야 할 질문들

- 실습문제 1. 요일 추가 하기
 
- 실습문제 2. 일자별(일별) 승하차총승객수를 구하시오.

- 실습문제 3. 2022.01월 ~ 12월 중 언제 지하철을 가장 많이 이용했을까? (기준: 승하차총승객수)

- 실습문제 4. 어느 요일에 가장 지하철 승객 수가 많을까? (기준: 승하차총승객수, 2022년 전체)

- 실습문제 5. 가장 승객이 많이 타는 승차역은?

- 실습문제 6. 노선별로 역별/요일별 승차승객수를 구하시오. (pivot_table)

- 실습문제 7. 1호선에서 가장 하차를 많이 하는 역은? (groupby)

- 실습문제 8. 2호선중에서 어느 역에서 승차가 가장 많이 발생할까?

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [89]:
import warnings
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)

In [90]:
subway = pd.read_csv('./서울시 지하철호선별 역별 승하차 인원 정보.csv', encoding = 'cp949', index_col = False) # 인덱스 컬럼에 대한 설정 : False
subway

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20220101,3호선,수서,7370,7076,20220104
1,20220101,3호선,학여울,461,473,20220104
2,20220101,3호선,대청,3224,2903,20220104
3,20220101,3호선,일원,3321,2803,20220104
4,20220101,경원선,창동,1,0,20220104
...,...,...,...,...,...,...
219740,20221231,4호선,신용산,11025,12099,20230103
219741,20221231,경부선,석수,5643,5147,20230103
219742,20221231,경부선,금천구청,7519,7426,20230103
219743,20221231,경부선,가산디지털단지,5579,7299,20230103


In [91]:
# 중복값 제거 
subway.drop_duplicates(inplace=True, keep='first')
# 해당 row의 데이터가 모두 똑같을때 제거됩니다
subway

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20220101,3호선,수서,7370,7076,20220104
1,20220101,3호선,학여울,461,473,20220104
2,20220101,3호선,대청,3224,2903,20220104
3,20220101,3호선,일원,3321,2803,20220104
4,20220101,경원선,창동,1,0,20220104
...,...,...,...,...,...,...
219740,20221231,4호선,신용산,11025,12099,20230103
219741,20221231,경부선,석수,5643,5147,20230103
219742,20221231,경부선,금천구청,7519,7426,20230103
219743,20221231,경부선,가산디지털단지,5579,7299,20230103


* 중복값 X

In [23]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219745 entries, 0 to 219744
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   사용일자    219745 non-null  int64 
 1   노선명     219745 non-null  object
 2   역명      219745 non-null  object
 3   승차총승객수  219745 non-null  int64 
 4   하차총승객수  219745 non-null  int64 
 5   등록일자    219745 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 11.7+ MB


In [24]:
subway['사용일자'] = pd.to_datetime(subway['사용일자'], format = '%Y%m%d')
# subway['사용일자'] = pd.to_datetime(subway['사용일자'])

In [25]:
subway['사용일자'].info()

<class 'pandas.core.series.Series'>
Int64Index: 219745 entries, 0 to 219744
Series name: 사용일자
Non-Null Count   Dtype         
--------------   -----         
219745 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.4 MB


In [26]:
subway['요일'] = subway['사용일자'].dt.dayofweek

In [27]:
temp = {
    0 : '월',
    1 : '화',
    2 : '수',
    3 : '목',
    4 : '금',
    5 : '토',
    6 : '일'
}

In [28]:
subway['요일'] = subway['요일'].map(temp)

In [43]:
subway

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일
0,2022-01-01,3호선,수서,7370,7076,20220104,토
1,2022-01-01,3호선,학여울,461,473,20220104,토
2,2022-01-01,3호선,대청,3224,2903,20220104,토
3,2022-01-01,3호선,일원,3321,2803,20220104,토
4,2022-01-01,경원선,창동,1,0,20220104,토
...,...,...,...,...,...,...,...
219740,2022-12-31,4호선,신용산,11025,12099,20230103,토
219741,2022-12-31,경부선,석수,5643,5147,20230103,토
219742,2022-12-31,경부선,금천구청,7519,7426,20230103,토
219743,2022-12-31,경부선,가산디지털단지,5579,7299,20230103,토


In [30]:
# 몇 월인지 column 추가 하기
subway['월'] = subway['사용일자'].dt.month

In [36]:
# 일자별(일별) 승하차 총 승객수 구하기
# 총 승객수 = 승차 총 승객수 + 하차 총 승객수
subway['총승객수'] = subway['승차총승객수'] + subway['하차총승객수']
subway.groupby('사용일자', as_index = False)['총승객수'].sum().sort_values('총승객수', ascending = False)

,사용일자,총승객수
300,2022-10-28,16103816
279,2022-10-07,16086459
272,2022-09-30,15872612
286,2022-10-14,15870097
293,2022-10-21,15856919
...,...,...
252,2022-09-10,5229907
0,2022-01-01,5207474
32,2022-02-02,5179270
30,2022-01-31,4214014


In [37]:
# 어느 월에 가장 지하철 승객 수가 많았을까요?
subway.groupby('월', as_index = False)['총승객수'].sum().sort_values('총승객수', ascending = False)

,월,총승객수
11,12,411586147
9,10,409364141
10,11,408088532
4,5,406148909
6,7,386418440
8,9,384701901
5,6,382106244
7,8,372175596
3,4,367720014
0,1,335012088


In [133]:
data_month = subway.pivot_table(index = '월', values = '총승객수', aggfunc='sum')
data_month.sort_values(by = '총승객수', ascending = False)

,총승객수
월,
12,411586147
10,409364141
11,408088532
5,406148909
7,386418440
9,384701901
6,382106244
8,372175596
4,367720014


In [56]:
# 어느 요일에 가장 지하철 승객 수가 많았을까요?
subway.groupby('요일', as_index = False)['총승객수'].sum().sort_values('총승객수', ascending = False)

,요일,총승객수
0,금,744278121
1,목,724134269
2,수,711512006
6,화,707603254
3,월,663101935
5,토,536198098
4,일,387104375


In [121]:
# 가장 승객이 많이 타는 승차역은?
subway.groupby('역명', as_index = False)['승차총승객수'].sum().sort_values('승차총승객수', ascending = False)

,역명,승차총승객수
430,잠실(송파구청),30095803
257,서울역,26640701
35,고속터미널,26365609
11,강남,26144333
517,홍대입구,25588832
...,...,...
68,굴포천,36
482,춘의,35
217,부평구청,32
230,삼산체육관,20


In [134]:
# 노선별로 역별/요일별 승차승객수를 구하시오.(pivot_table)

pv_month = pd.pivot_table(subway,                # 피벗할 데이터프레임
                     index = ['노선명','역명'],    # 행 위치에 들어갈 열
                     columns = '요일',    # 열 위치에 들어갈 열 > 사용한 컬럼 없을 땐 제외해도 가능
                     values = '승차총승객수',     # 데이터로 사용할 열
                     aggfunc = 'sum')   # 데이터 집계함수 > default : mean
pv_month

요일               금          목          수          월          일          토  \
노선명 역명                                                                      
1호선 동대문   586678.0   566508.0   560116.0   543429.0   493857.0   554052.0   
    동묘앞   432191.0   407326.0   403143.0   378809.0   597602.0   581224.0   
    서울역  2661153.0  2429461.0  2340700.0  2296132.0  1561413.0  1954721.0   
    시청   1265284.0  1269250.0  1245253.0  1141914.0   492339.0   804116.0   
    신설동   751771.0   724587.0   711780.0   668970.0   394963.0   519406.0   
...            ...        ...        ...        ...        ...        ...   
중앙선 원덕     19862.0    18874.0    19257.0    19882.0    20575.0    22513.0   
    중랑    309184.0   299326.0   295219.0   279579.0   179709.0   240169.0   
    지평      3452.0     3529.0     3381.0     3580.0     3214.0     2968.0   
    팔당     48033.0    45755.0    45995.0    44290.0    62962.0    69583.0   
    회기   1401511.0  1378198.0  1342690.0  1253740.0   837620.0  1110203.0   

요일               화  
노선명 역명              
1호선 동대문   558567.0  
    동묘앞   396517.0  
    서울역  2332031.0  
    시청   1254394.0  
    신설동   698462.0  
...            ...  
중앙선 원덕     19369.0  
    중랑    295347.0  
    지평      3438.0  
    팔당     44466.0  
    회기   1336649.0  

[622 rows x 7 columns]

In [135]:
pv_month.columns

Index(['금', '목', '수', '월', '일', '토', '화'], dtype='object', name='요일')

In [137]:
# 노선별로 역별/요일별 승차승객수를 구하시오.(pivot_table) -요일 정리
pv_month_sort = pv_month[['월','화','수','목','금','토','일']]
pv_month_sort

요일               월          화          수          목          금          토  \
노선명 역명                                                                      
1호선 동대문   543429.0   558567.0   560116.0   566508.0   586678.0   554052.0   
    동묘앞   378809.0   396517.0   403143.0   407326.0   432191.0   581224.0   
    서울역  2296132.0  2332031.0  2340700.0  2429461.0  2661153.0  1954721.0   
    시청   1141914.0  1254394.0  1245253.0  1269250.0  1265284.0   804116.0   
    신설동   668970.0   698462.0   711780.0   724587.0   751771.0   519406.0   
...            ...        ...        ...        ...        ...        ...   
중앙선 원덕     19882.0    19369.0    19257.0    18874.0    19862.0    22513.0   
    중랑    279579.0   295347.0   295219.0   299326.0   309184.0   240169.0   
    지평      3580.0     3438.0     3381.0     3529.0     3452.0     2968.0   
    팔당     44290.0    44466.0    45995.0    45755.0    48033.0    69583.0   
    회기   1253740.0  1336649.0  1342690.0  1378198.0  1401511.0  1110203.0   

요일               일  
노선명 역명              
1호선 동대문   493857.0  
    동묘앞   597602.0  
    서울역  1561413.0  
    시청    492339.0  
    신설동   394963.0  
...            ...  
중앙선 원덕     20575.0  
    중랑    179709.0  
    지평      3214.0  
    팔당     62962.0  
    회기    837620.0  

[622 rows x 7 columns]

In [138]:
# 2호선 역/요일별 총 승차 승객수
# pivot_table 활용
line_2 = subway[subway['노선명']=='2호선']

In [140]:
line_2_pivot = line_2.pivot_table(index = '역명', columns = '요일', values = '승차총승객수', aggfunc = 'sum')
line_2_pivot = line_2_pivot[['월', '화', '수', '목', '금', '토', '일']]
line_2_pivot

요일,월,화,수,목,금,토,일
역명,,,,,,,
강남,3944294,4247525,4226305,4339223,4560662,3061360,1764964
강변(동서울터미널),1547838,1551439,1566582,1612265,1774188,1485299,1181725
건대입구,1684530,1767660,1803044,1847160,1975477,1827640,1370591
교대(법원.검찰청),1636589,1799272,1788731,1824649,1835665,911099,547216
구로디지털단지,2882284,3097899,3081809,3148951,3162019,1819810,1258063
구의(광진구청),1151041,1239890,1241682,1263279,1287034,871900,593516
낙성대(강감찬),1321711,1418798,1435431,1459673,1508896,1135048,816603
당산,883616,962498,968115,984688,996778,673066,454304
대림(구로구청),1190525,1238106,1241112,1252201,1271117,948829,788477


In [141]:
# 1호선에서 가장 하차를 많이 하는 역은? (groupby)
subway[subway['노선명'] == '1호선'].groupby('역명', as_index = False)['하차총승객수'].sum().sort_values('하차총승객수', ascending = False)

,역명,하차총승객수
2,서울역,15107638
6,종각,10940702
7,종로3가,7913074
3,시청,7549153
8,종로5가,7463686
9,청량리(서울시립대입구),6752687
5,제기동,5839885
4,신설동,4326111
0,동대문,3680871
1,동묘앞,3317739


In [142]:
# 2호선중에서 어느 역에서 승차가 가장 많이 발생할까?
subway[subway['노선명'] == '2호선'].groupby('역명', as_index = False)['승차총승객수'].sum().sort_values('승차총승객수', ascending = False)

,역명,승차총승객수
0,강남,26144333
42,잠실(송파구청),25040626
49,홍대입구,20953932
27,신림,19946559
4,구로디지털단지,18450835
20,선릉,16891665
16,삼성(무역센터),16806757
26,신도림,15966447
18,서울대입구(관악구청),15608934
33,역삼,15238648


In [143]:
# 2호선중에서 어느 역에서 하차가 가장 많이 발생할까?
subway[subway['노선명'] == '2호선'].groupby('역명', as_index = False)['하차총승객수'].sum().sort_values('하차총승객수', ascending = False)

,역명,하차총승객수
0,강남,25620317
42,잠실(송파구청),24620229
49,홍대입구,22300019
27,신림,19238017
4,구로디지털단지,18369622
33,역삼,17100099
16,삼성(무역센터),16877761
26,신도림,15974956
18,서울대입구(관악구청),15273661
20,선릉,14977748
